In [1]:
import os, sys
import unidecode
import re
import json
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Ce notebook analyse les réponses aux questions en cherchant des liens entre les mots les plus utilisés, dans le but de comprendre les demandes les plus récurrentes des francais.

Dans un premier temps, les réponses sont préprocessées et transformées en liste de mots-clés. Puis, pour chaque réponse, úne analyse basique de la proximité des mots est réalisée. Bien que simple, cette methode permet de faire apparaitre des thèmes récurrents et de comprendre la teneur générale de ces demandes.

Les trois métriques utilisées sont :
+ co-occurence des mots : $co-occurence(i, j) = \sum_{users} 1_{i \in user} 1_{j \in user}$
+ confiance : $confiance(i -> j) = P(j | i)$
+ lift : $lift(i, j) = \frac{P(i \cap j)}{P(i)P(j)}$

La co-occurence permet d'évaluer les mots présents de manière commune, et donne une première idée des thèmes généraux.

La confiance donne une idée de la présence de $j$ quand $i$ est déjà présent : il surpondère les mots peu présents par rapport à la co-occurence.

Le lift estime le surplus de présence commune de $i$ et $j$ par rapport au cas où $i$ et $j$ sont indépendants. Cela permet de trouver des relations de causalité plus intéressantes, notamment dans des cas où certains mots sont peu présents mais très liés.

Pour chaque question :
+ On récupère les mots les plus employés sur ce thème
+ Pour chacun de ces mots, on regarde pour chaque métrique les mots les plus liés, ce qui donne une bonne idée des liens et des demandes

On peut ensuite raffiner l'analyse sur des mots moins présents mais porteurs de sens.

In [2]:
# function definitions

def compute_similarities(question_name, df_res):
    qu = df_res[question_name].dropna()
    df_qu_one_hot = compute_one_hot(qu)
    df_cov = compute_covariance(df_qu_one_hot)
    df_confidence = compute_confidence(df_cov)
    df_lift = compute_lift(df_cov, df_qu_one_hot)
    
    return qu, df_qu_one_hot, df_cov, df_confidence, df_lift
    
def compute_one_hot(qu):
    qu_words = []
    for i in qu:
        qu_words.extend(i)
    qu_words = list(set(qu_words))
    dic_qu_words = {word:i for i, word in enumerate(qu_words)}
    dic_qu_words_inverse = {i:word for i, word in enumerate(qu_words)}
    
    qu_one_hot = np.zeros((len(qu), len(qu_words)), dtype=bool)
    for i, li_word in enumerate(qu):
        for word in li_word:
            qu_one_hot[i, dic_qu_words[word]] = True
            
    df_qu_one_hot = pd.DataFrame(columns=qu_words, data=qu_one_hot).to_sparse(fill_value=False)

    
    temp_mask = np.sum(df_qu_one_hot.values, axis=0, dtype=np.uint16) > 250
    words_enough = df_qu_one_hot.columns.values[temp_mask]
    df_qu_one_hot = df_qu_one_hot[words_enough].to_sparse(fill_value=0)
    
    del temp_mask
    del qu_one_hot
    gc.collect()
    
    return df_qu_one_hot

def compute_covariance(df_qu_one_hot):
    N = df_qu_one_hot.shape[1]
    
    cov = np.zeros((N, N), dtype=np.uint16)
    np.matmul(df_qu_one_hot.values.T, df_qu_one_hot.values, out=cov)
    
    df_cov = pd.DataFrame(columns=df_qu_one_hot.columns, index=df_qu_one_hot.columns, data=cov) * (cov > 100)
    del cov
    gc.collect()
    
    return df_cov

def compute_confidence(df_cov):
    df_confidence = np.diag(1 / np.diag(df_cov)).dot(df_cov).astype(np.float32)
    df_confidence = pd.DataFrame(index=df_cov.index, columns=df_cov.index, data=df_confidence)
    return df_confidence

def compute_lift(df_cov, df_qu_one_hot):
    inv_diag_cov = 1 / np.diag(df_cov.values)
    lift = inv_diag_cov.reshape((-1, 1)) * df_cov.values * inv_diag_cov.reshape((1, -1))
    df_lift = pd.DataFrame(columns=df_qu_one_hot.columns,
                             index=df_qu_one_hot.columns,
                             data=lift).replace(0.0, np.nan).astype(np.float32)
    del lift
    del inv_diag_cov
    gc.collect()
    
    return df_lift

# Loading of files and preprocessing

In [3]:
PATH_TO_FILE = "./proposals_cleaned/ecologie.json"

In [4]:
with open(PATH_TO_FILE, "r") as f:
    raw_json = [s.strip() for s in f.readlines()]
    
parsed_json = [json.loads(raw_json[i]) for i in range(len(raw_json))]
del raw_json

In [5]:
col = ["titre"]
for ques in parsed_json[0]["responses"]:
    col.append(ques["question"]["id"]) 
        
data = []
for elt in parsed_json:
    resp = {"titre" : elt["titre"]}
    for ques in elt["responses"]:
        try:
            text = json.loads((ques["value"]))
            resp[ques["question"]["title"]] = ''
            if text["labels"] != []:
                resp[ques["question"]["title"]] += ", ".join(text["labels"])
            if text["other"] is not None:
                resp[ques["question"]["title"]] += ", " + text["other"]
            if resp[ques["question"]["title"]] == '':
                resp[ques["question"]["title"]] = np.nan
        except:
            if ques["value"] is not None and ques["value"][0] != "{":
                resp[ques["question"]["title"]] = ques["value"]
            else:
                resp[ques["question"]["title"]] = np.nan
    data.append(resp)

In [6]:
res = pd.DataFrame.from_dict(data)
del data
gc.collect()

0

In [7]:
res.shape

(52430, 17)

In [8]:
0.910319 * 52430

47728.02517

In [9]:
res.notna().mean(axis=0).sort_values(ascending=False)

titre                                                                                                                                                                                                                                                  1.000000
Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?                                                                                                                                              0.946004
Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?                                                                                                                                                           0.911406
À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?                                                                                                                                                          

# Find important words

In [10]:
li_word_count = []
for elt in res.values:
    for item in elt:
        if type(item) == str:
            li_word_count.extend([unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item)])

In [11]:
dic_word_count = {}
for i in li_word_count:
    if len(i) > 0:
        if i in dic_word_count.keys():
            dic_word_count[i] +=1
        else:
            dic_word_count[i] = 1

In [12]:
df_word_count = pd.DataFrame.from_dict(dic_word_count, orient="index")
df_word_count.sort_values(0, ascending=False)
df_word_count.columns = ["freq_in_debate"]

In [13]:
del li_word_count
del dic_word_count

gc.collect()

202

In [14]:
df_word_count = df_word_count[df_word_count["freq_in_debate"] > 100]

In [15]:
li_stopwords = [unidecode.unidecode(x.strip().lower()) for x in open("french_stopwords.txt", 'r').readlines()]
li_stopwords = set(li_stopwords)
li_stopwords.remove("aucun")
li_stopwords.remove("tout")
li_stopwords.remove("tous")
li_stopwords.remove("non")

In [16]:
new_data = []
for i, elt in enumerate(res.index):
    temp_res = []
    for col in res.columns:
        item = res.loc[elt, col]
        if type(item) == str:
            li_item = [unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item) if unidecode.unidecode(x.lower()) not in li_stopwords]
            temp_res.append(li_item)
        else:
            temp_res.append(None)
    new_data.append(temp_res)

In [17]:
new_res = pd.DataFrame(new_data, columns=res.columns).to_sparse()
del new_data
gc.collect()

0

# Question 1 : Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?

In [18]:
question_1 = "Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?"
qu_1,df_qu_1_one_hot,df_cov_1,df_confidence_1,df_lift_1 = compute_similarities(question_1,new_res)

In [19]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_1[i].sort_values(ascending=False).head(10))

climatiques


climatiques     14449
dereglements    14339
secheresse      13853
crue            13844
pollution         389
biodiversite      373
air               343
disparition       228
especes           218
erosion           179
Name: climatiques, dtype: uint16

dereglements


dereglements    14431
climatiques     14339
secheresse      13849
crue            13843
pollution         378
biodiversite      371
air               329
disparition       215
especes           210
erosion           179
Name: dereglements, dtype: uint16

secheresse


...

lies               722
tous               494
problemes          354
importants         137
environnement        0
crue                 0
non                  0
surconsommation      0
vie                  0
dereglements         0
Name: lies, dtype: uint16

environnement


environnement      717
tout               163
tous               128
pollution          109
crue                 0
non                  0
importants           0
surconsommation      0
vie                  0
dereglements         0
Name: environnement, dtype: uint16

sols


sols               674
pollution          455
air                284
eau                213
biodiversite       127
reponses             0
environnement        0
non                  0
importants           0
surconsommation      0
Name: sols, dtype: uint16

In [20]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_1[i].sort_values(ascending=False).head(10))

climatiques


climatiques     1.000000
crue            0.999639
secheresse      0.997408
dereglements    0.993625
littoral        0.257426
erosion         0.240591
tout            0.046196
biodiversite    0.037052
pollution       0.030812
air             0.030223
Name: climatiques, dtype: float32

dereglements


dereglements    1.000000
crue            0.999567
secheresse      0.997120
climatiques     0.992387
littoral        0.245875
erosion         0.240591
biodiversite    0.036853
pollution       0.029941
air             0.028989
disparition     0.022494
Name: dereglements, dtype: float32

secheresse


...

lies               1.000000
problemes          0.252857
importants         0.206637
tous               0.192518
environnement      0.000000
crue               0.000000
non                0.000000
surconsommation    0.000000
vie                0.000000
dereglements       0.000000
Name: lies, dtype: float32

environnement


environnement      1.000000
tout               0.073822
tous               0.049883
pollution          0.008634
crue               0.000000
non                0.000000
importants         0.000000
surconsommation    0.000000
vie                0.000000
dereglements       0.000000
Name: environnement, dtype: float32

sols


sols               1.000000
eau                0.208824
pollution          0.036040
air                0.025024
biodiversite       0.012615
reponses           0.000000
environnement      0.000000
non                0.000000
importants         0.000000
surconsommation    0.000000
Name: sols, dtype: float32

In [21]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_1[i].sort_values(ascending=False).head(10))

climatiques


climatiques     0.000069
crue            0.000069
secheresse      0.000069
dereglements    0.000069
littoral        0.000018
erosion         0.000017
tout            0.000003
biodiversite    0.000003
pollution       0.000002
air             0.000002
Name: climatiques, dtype: float32

dereglements


dereglements    0.000069
crue            0.000069
secheresse      0.000069
climatiques     0.000069
littoral        0.000017
erosion         0.000017
biodiversite    0.000003
pollution       0.000002
air             0.000002
disparition     0.000002
Name: dereglements, dtype: float32

secheresse


...

lies           0.001385
problemes      0.000350
importants     0.000286
tous           0.000267
terre               NaN
energie             NaN
faut                NaN
climatiques         NaN
secheresse          NaN
agriculture         NaN
Name: lies, dtype: float32

environnement


environnement    0.001395
tout             0.000103
tous             0.000070
pollution        0.000012
terre                 NaN
energie               NaN
lies                  NaN
faut                  NaN
climatiques           NaN
secheresse            NaN
Name: environnement, dtype: float32

sols


sols            0.001484
eau             0.000310
pollution       0.000053
air             0.000037
biodiversite    0.000019
terre                NaN
energie              NaN
lies                 NaN
faut                 NaN
climatiques          NaN
Name: sols, dtype: float32

In [22]:
gc.collect()

285

# Question 2 : Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?

In [23]:
question_2 = "Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?"
qu_2,df_qu_2_one_hot,df_cov_2,df_confidence_2,df_lift_2 = compute_similarities(question_2,new_res)

In [24]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_2[i].sort_values(ascending=False).head(10))

faire


faire          6889
tout           1675
faut           1647
tous           1578
produits       1364
entreprises    1213
non            1182
france         1179
exemple        1157
mettre         1040
Name: faire, dtype: uint16

produits


produits        5947
tous            1475
taxer           1448
interdire       1419
faire           1364
agriculture     1225
consommation    1207
non             1192
entreprises     1180
tout            1158
Name: produits, dtype: uint16

tous


...

exemple        3630
faire          1157
faut            978
tout            974
produits        947
tous            939
non             784
france          751
transports      741
entreprises     733
Name: exemple, dtype: uint16

favoriser


favoriser      3612
transports     1076
produits       1003
agriculture     776
taxer           707
entreprises     690
faire           689
transport       674
developper      673
tous            649
Name: favoriser, dtype: uint16

agriculture


agriculture    3602
produits       1225
bio             972
pesticides      906
favoriser       776
interdire       763
tous            746
faire           745
tout            686
biologique      686
Name: agriculture, dtype: uint16

In [25]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_2[i].sort_values(ascending=False).head(10))

faire


faire         1.000000
sorte         0.797647
payer         0.646586
croire        0.616725
savoir        0.561828
comprendre    0.543253
pourra        0.538941
chose         0.516765
veut          0.506329
veulent       0.492063
Name: faire, dtype: float32

produits


produits           1.000000
importes           0.955711
phytosanitaires    0.891720
issus              0.755906
nocifs             0.737864
toxiques           0.716292
chimiques          0.665698
importation        0.651219
alimentaires       0.630357
tva                0.613197
Name: produits, dtype: float32

tous


...

exemple       1.000000
montrer       0.722527
titre         0.364261
6             0.323450
pourraient    0.322034
or            0.296196
ensuite       0.293722
5             0.280423
cas           0.279831
point         0.277419
Name: exemple, dtype: float32

favoriser


favoriser      1.000000
courts         0.411576
circuits       0.409651
proximite      0.345304
covoiturage    0.325444
favorisant     0.317073
locale         0.313280
locaux         0.289871
biologique     0.288973
elevage        0.283262
Name: favoriser, dtype: float32

agriculture


agriculture        1.000000
biologique         0.869455
intensive          0.822414
raisonnee          0.721402
respectueuse       0.672241
permaculture       0.534247
elevage            0.515021
exploitations      0.498371
agro               0.437909
phytosanitaires    0.423567
Name: agriculture, dtype: float32

In [26]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_2[i].sort_values(ascending=False).head(10))

faire


faire         0.000145
sorte         0.000116
payer         0.000094
croire        0.000090
savoir        0.000082
comprendre    0.000079
pourra        0.000078
chose         0.000075
veut          0.000073
veulent       0.000071
Name: faire, dtype: float32

produits


produits           0.000168
importes           0.000161
phytosanitaires    0.000150
issus              0.000127
nocifs             0.000124
toxiques           0.000120
chimiques          0.000112
importation        0.000110
alimentaires       0.000106
tva                0.000103
Name: produits, dtype: float32

tous


...

exemple       0.000275
montrer       0.000199
titre         0.000100
6             0.000089
pourraient    0.000089
or            0.000082
ensuite       0.000081
5             0.000077
cas           0.000077
point         0.000076
Name: exemple, dtype: float32

favoriser


favoriser      0.000277
courts         0.000114
circuits       0.000113
proximite      0.000096
covoiturage    0.000090
favorisant     0.000088
locale         0.000087
locaux         0.000080
biologique     0.000080
elevage        0.000078
Name: favoriser, dtype: float32

agriculture


agriculture        0.000278
biologique         0.000241
intensive          0.000228
raisonnee          0.000200
respectueuse       0.000187
permaculture       0.000148
elevage            0.000143
exploitations      0.000138
agro               0.000122
phytosanitaires    0.000118
Name: agriculture, dtype: float32

In [27]:
gc.collect()

98

# Question 3 : Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?

In [28]:
question_3 = "Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?"
qu_3,df_qu_3_one_hot,df_cov_3,df_confidence_3,df_lift_3 = compute_similarities(question_3, new_res)

In [29]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_3[i].sort_values(ascending=False).head(10))

oui


oui    31023
non        0
Name: oui, dtype: uint16

non


non    16762
oui        0
Name: non, dtype: uint16

In [30]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_3[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [31]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_3[i].sort_values(ascending=False).head(10))

oui


oui    0.000032
non         NaN
Name: oui, dtype: float32

non


non    0.00006
oui        NaN
Name: non, dtype: float32

In [32]:
gc.collect()

14

# Question 4 : Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?

In [33]:
question_4 = "Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?"
qu_4,df_qu_4_one_hot,df_cov_4,df_confidence_4,df_lift_4 = compute_similarities(question_4, new_res)

In [34]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_4[i].sort_values(ascending=False).head(10))

pollution


pollution      4978
air            2400
eau             453
climatique      417
disparition     385
sante           383
secheresse      371
climatiques     314
ville           309
especes         296
Name: pollution, dtype: uint16

air


air            3696
pollution      2400
qualite         631
eau             498
pollue          315
climatique      288
sante           276
disparition     275
secheresse      251
ville           243
Name: air, dtype: uint16

secheresse


...

canicules      1272
inondations     187
tempetes        178
frequentes      168
pollution       167
secheresses     135
climatiques     115
air             110
chauffage         0
produits          0
Name: canicules, dtype: uint16

sante


sante            1267
pollution         383
air               276
problemes         252
impact            123
environnement       0
dereglements        0
chaud               0
caniculaires        0
produits            0
Name: sante, dtype: uint16

climat


climat           1253
pollution         196
changement        184
air               120
change            104
dereglement       102
saisons           102
environnement       0
dereglements        0
chaud               0
Name: climat, dtype: uint16

In [35]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_4[i].sort_values(ascending=False).head(10))

pollution


pollution        1.000000
atmospherique    0.945017
air              0.649351
pics             0.589977
villes           0.492929
paris            0.444056
sols             0.434010
pesticides       0.430939
ville            0.420408
grandes          0.369565
Name: pollution, dtype: float32

air


air             1.000000
pollue          0.779703
qualite         0.747630
pollution       0.482121
paris           0.409091
pesticides      0.386740
alimentation    0.370466
sols            0.352792
villes          0.343434
ville           0.330612
Name: air, dtype: float32

secheresse


...

canicules       1.000000
frequentes      0.222812
secheresses     0.154286
tempetes        0.125529
inondations     0.104645
climatiques     0.054554
pollution       0.033548
air             0.029762
chauffage       0.000000
caniculaires    0.000000
Name: canicules, dtype: float32

sante


sante            1.000000
problemes        0.329843
impact           0.170360
pollution        0.076939
air              0.074675
environnement    0.000000
dereglements     0.000000
chaud            0.000000
caniculaires     0.000000
produits         0.000000
Name: sante, dtype: float32

climat


climat           1.000000
change           0.383764
changement       0.110777
dereglement      0.088542
saisons          0.042114
pollution        0.039373
air              0.032468
environnement    0.000000
dereglements     0.000000
chaud            0.000000
Name: climat, dtype: float32

In [36]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_4[i].sort_values(ascending=False).head(10))

pollution


pollution        0.000201
atmospherique    0.000190
air              0.000130
pics             0.000119
villes           0.000099
paris            0.000089
sols             0.000087
pesticides       0.000087
ville            0.000084
grandes          0.000074
Name: pollution, dtype: float32

air


air             0.000271
pollue          0.000211
qualite         0.000202
pollution       0.000130
paris           0.000111
pesticides      0.000105
alimentation    0.000100
sols            0.000095
villes          0.000093
ville           0.000089
Name: air, dtype: float32

secheresse


...

canicules      0.000786
frequentes     0.000175
secheresses    0.000121
tempetes       0.000099
inondations    0.000082
climatiques    0.000043
pollution      0.000026
air            0.000023
frequents           NaN
petits              NaN
Name: canicules, dtype: float32

sante


sante        0.000789
problemes    0.000260
impact       0.000134
pollution    0.000061
air          0.000059
frequents         NaN
petits            NaN
cout              NaN
froid             NaN
difficile         NaN
Name: sante, dtype: float32

climat


climat         0.000798
change         0.000306
changement     0.000088
dereglement    0.000071
saisons        0.000034
pollution      0.000031
air            0.000026
frequents           NaN
petits              NaN
cout                NaN
Name: climat, dtype: float32

In [37]:
gc.collect()

147

# Question 5 :  À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?

In [38]:
question_5 = "À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?"
qu_5,df_qu_5_one_hot,df_cov_5,df_confidence_5,df_lift_5 = compute_similarities(question_5, new_res)

In [39]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_5[i].sort_values(ascending=False).head(10))

oui


oui    44761
non        0
Name: oui, dtype: uint16

non


non    2967
oui       0
Name: non, dtype: uint16

In [40]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_5[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [41]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_5[i].sort_values(ascending=False).head(10))

oui


oui    0.000022
non         NaN
Name: oui, dtype: float32

non


non    0.000337
oui         NaN
Name: non, dtype: float32

In [42]:
gc.collect()

14

# Question 6 :  Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?

In [43]:
question_6 = "Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?"
qu_6,df_qu_6_one_hot,df_cov_6,df_confidence_6,df_lift_6 = compute_similarities(question_6, new_res)

In [44]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_6[i].sort_values(ascending=False).head(10))

dechets


dechets         13962
tri              5985
produits         3330
voiture          3189
consommation     3100
eau              2747
trie             2453
bio              2450
chauffage        2030
transports       1903
Name: dechets, dtype: uint16

tri


tri             12166
dechets          5985
selectif         4971
consommation     2516
produits         2455
voiture          2376
eau              2272
utilisation      1879
chauffage        1865
bio              1707
Name: tri, dtype: uint16

voiture


...

local           3388
bio             1500
dechets         1221
produits         876
voiture          847
consommer        841
consommation     798
acheter          727
achete           664
tri              611
Name: local, dtype: uint16

viande


viande          3274
consommation    1695
produits        1225
dechets         1172
bio             1169
voiture          938
tri              691
mange            676
velo             655
eau              619
Name: viande, dtype: uint16

trie


trie            3168
dechets         2453
voiture          849
produits         777
utilise          695
achete           655
eau              644
bio              633
consommation     538
commun           442
Name: trie, dtype: uint16

In [45]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_6[i].sort_values(ascending=False).head(10))

dechets


dechets      1.000000
trier        0.795552
ramassage    0.781690
trie         0.774306
verts        0.754658
ramasse      0.747692
vegetaux     0.685606
gestion      0.675393
menagers     0.659062
composte     0.625641
Name: dechets, dtype: float32

tri


tri           1.000000
selectif      0.968628
menageres     0.656028
ordures       0.614424
respect       0.538265
compostage    0.524862
ramassage     0.503521
economie      0.494737
limitation    0.469981
poubelles     0.463796
Name: tri, dtype: float32

voiture


...

local        1.000000
manger       0.316856
consommer    0.309191
mange        0.285290
consomme     0.284427
acheter      0.244616
saison       0.242718
occasion     0.240048
bio          0.234632
achete       0.233968
Name: local, dtype: float32

viande


viande     1.000000
poisson    0.912281
mange      0.502229
semaine    0.481390
manger     0.411669
reduit     0.347732
issus      0.327161
qualite    0.283505
reduis     0.266026
fruits     0.252014
Name: viande, dtype: float32

trie


trie         1.000000
economise    0.330952
jette        0.314587
composte     0.296154
roule        0.240244
achete       0.230796
mange        0.226597
evite        0.226532
utilise      0.222329
chauffe      0.217749
Name: trie, dtype: float32

In [46]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_6[i].sort_values(ascending=False).head(10))

dechets


dechets      0.000072
trier        0.000057
ramassage    0.000056
trie         0.000055
verts        0.000054
ramasse      0.000054
vegetaux     0.000049
gestion      0.000048
menagers     0.000047
composte     0.000045
Name: dechets, dtype: float32

tri


tri           0.000082
selectif      0.000080
menageres     0.000054
ordures       0.000051
respect       0.000044
compostage    0.000043
ramassage     0.000041
economie      0.000041
limitation    0.000039
poubelles     0.000038
Name: tri, dtype: float32

voiture


...

local        0.000295
manger       0.000094
consommer    0.000091
mange        0.000084
consomme     0.000084
acheter      0.000072
saison       0.000072
occasion     0.000071
bio          0.000069
achete       0.000069
Name: local, dtype: float32

viande


viande     0.000305
poisson    0.000279
mange      0.000153
semaine    0.000147
manger     0.000126
reduit     0.000106
issus      0.000100
qualite    0.000087
reduis     0.000081
fruits     0.000077
Name: viande, dtype: float32

trie


trie         0.000316
economise    0.000104
jette        0.000099
composte     0.000093
roule        0.000076
achete       0.000073
mange        0.000072
evite        0.000072
utilise      0.000070
chauffe      0.000069
Name: trie, dtype: float32

In [47]:
gc.collect()

105

# Question 7 : Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?

In [48]:
question_7 = "Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?"
qu_7,df_qu_7_one_hot,df_cov_7,df_confidence_7,df_lift_7 = compute_similarities(question_7, new_res)

In [49]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_7[i].sort_values(ascending=False).head(10))

transports


transports    7063
commun        4463
voiture       1224
chauffage      909
communs        722
developper     683
cyclables      639
bus            622
vehicule       617
prendre        614
Name: transports, dtype: uint16

commun


commun        5907
transports    4463
transport     1681
voiture       1116
chauffage      794
vehicule       572
prendre        545
developper     541
faire          528
cyclables      519
Name: commun, dtype: uint16

voiture


...

changer         2008
chauffage        539
voiture          395
faire            362
chaudiere        334
vehicule         312
faut             302
aides            290
comportement     287
transports       285
Name: changer, dtype: uint16

tous


tous          1877
transports     388
faire          372
voiture        338
chauffage      306
commun         292
faut           273
tout           266
exemple        219
prix           195
Name: tous, dtype: uint16

isolation


isolation     1795
chauffage      634
transports     427
aides          399
commun         328
voiture        310
travaux        288
aide           274
faire          258
maison         228
Name: isolation, dtype: uint16

In [50]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_7[i].sort_values(ascending=False).head(10))

transports


transports    1.000000
communs       0.912769
frequents     0.795699
gratuits      0.793515
gratuite      0.788851
commun        0.755544
collectifs    0.752711
frequence     0.692015
publics       0.645833
adaptes       0.637874
Name: transports, dtype: float32

commun


commun        1.000000
transports    0.631884
gratuite      0.587838
transport     0.571574
frequents     0.555556
frequence     0.520913
horaires      0.488753
gratuits      0.481229
reseau        0.460882
adaptes       0.448505
Name: commun, dtype: float32

voiture


...

changer          1.000000
fenetres         0.307479
comportement     0.274904
chaudiere        0.252839
mode             0.244048
comportements    0.243682
diesel           0.232283
inciter          0.204152
ans              0.162416
moyens           0.145974
Name: changer, dtype: float32

tous


tous            1.000000
jours           0.604895
ans             0.225503
produits        0.175214
citoyens        0.168992
information     0.139946
monde           0.139142
consommation    0.133779
entreprises     0.123158
mettre          0.123138
Name: tous, dtype: float32

isolation


isolation    1.000000
thermique    0.469965
travaux      0.435703
logements    0.360920
fenetres     0.318560
logement     0.264451
maison       0.213683
ameliorer    0.204108
meilleure    0.192604
aides        0.174160
Name: isolation, dtype: float32

In [51]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_7[i].sort_values(ascending=False).head(10))

transports


transports    0.000142
communs       0.000129
frequents     0.000113
gratuits      0.000112
gratuite      0.000112
commun        0.000107
collectifs    0.000107
frequence     0.000098
publics       0.000091
adaptes       0.000090
Name: transports, dtype: float32

commun


commun        0.000169
transports    0.000107
gratuite      0.000100
transport     0.000097
frequents     0.000094
frequence     0.000088
horaires      0.000083
gratuits      0.000081
reseau        0.000078
adaptes       0.000076
Name: commun, dtype: float32

voiture


...

changer          0.000498
fenetres         0.000153
comportement     0.000137
chaudiere        0.000126
mode             0.000122
comportements    0.000121
diesel           0.000116
inciter          0.000102
ans              0.000081
moyens           0.000073
Name: changer, dtype: float32

tous


tous            0.000533
jours           0.000322
ans             0.000120
produits        0.000093
citoyens        0.000090
information     0.000075
monde           0.000074
consommation    0.000071
entreprises     0.000066
mettre          0.000066
Name: tous, dtype: float32

isolation


isolation    0.000557
thermique    0.000262
travaux      0.000243
logements    0.000201
fenetres     0.000177
logement     0.000147
maison       0.000119
ameliorer    0.000114
meilleure    0.000107
aides        0.000097
Name: isolation, dtype: float32

In [52]:
gc.collect()

105

# Question 8 : Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?

In [53]:
question_8 = "Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?"
qu_8,df_qu_8_one_hot,df_cov_8,df_confidence_8,df_lift_8 = compute_similarities(question_8, new_res)

In [54]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_8[i].sort_values(ascending=False).head(10))

transports


transports    3705
commun        2249
gratuite       469
prix           450
voiture        402
developper     392
gratuits       316
communs        316
publics        304
produits       274
Name: transports, dtype: uint16

faire


faire       3138
faut         474
tout         464
changer      384
prix         343
voiture      339
exemple      333
payer        331
tous         302
produits     298
Name: faire, dtype: uint16

produits


...

vehicule      1620
electrique     570
achat          424
aide           316
changer        251
prix           227
vehicules      224
propre         216
diesel         189
aides          187
Name: vehicule, dtype: uint16

vehicules


vehicules      1599
electriques     487
prix            367
electrique      251
achat           248
vehicule        224
polluants       224
propres         221
transports      192
aides           183
Name: vehicules, dtype: uint16

taxes


taxes        1595
produits      350
impots        164
faire         151
tout          148
augmenter     148
baisser       146
prix          145
exemple       144
tous          143
Name: taxes, dtype: uint16

In [55]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_8[i].sort_values(ascending=False).head(10))

transports


transports    1.000000
communs       0.908046
commun        0.766008
gratuits      0.747045
gratuite      0.714939
collectifs    0.640741
publics       0.590291
trains        0.402827
bus           0.392581
villes        0.370940
Name: transports, dtype: float32

faire


faire        1.000000
economies    0.384127
efforts      0.376884
payer        0.364136
effort       0.343750
choix        0.281623
gros         0.270455
planete      0.261538
mieux        0.257745
aller        0.254274
Name: faire, dtype: float32

produits


...

vehicule      1.000000
hybride       0.522293
propre        0.396330
polluant      0.394541
diesel        0.323630
electrique    0.311475
achat         0.276762
prime         0.240000
acheter       0.223192
essence       0.215488
Name: vehicule, dtype: float32

vehicules


vehicules        1.000000
electriques      0.434821
propres          0.399638
constructeurs    0.392491
hydrogene        0.326087
polluants        0.325581
diesel           0.210616
essence          0.171717
achat            0.161880
vehicule         0.138272
Name: vehicules, dtype: float32

taxes


taxes         1.000000
carburants    0.297688
augmenter     0.167991
baisser       0.159912
taxe          0.118968
transition    0.117442
impots        0.115412
produits      0.114267
taxer         0.082650
ecologique    0.082360
Name: taxes, dtype: float32

In [56]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_8[i].sort_values(ascending=False).head(10))

transports


transports    0.000270
communs       0.000245
commun        0.000207
gratuits      0.000202
gratuite      0.000193
collectifs    0.000173
publics       0.000159
trains        0.000109
bus           0.000106
villes        0.000100
Name: transports, dtype: float32

faire


faire        0.000319
economies    0.000122
efforts      0.000120
payer        0.000116
effort       0.000110
choix        0.000090
gros         0.000086
planete      0.000083
mieux        0.000082
aller        0.000081
Name: faire, dtype: float32

produits


...

vehicule      0.000617
hybride       0.000322
propre        0.000245
polluant      0.000244
diesel        0.000200
electrique    0.000192
achat         0.000171
prime         0.000148
acheter       0.000138
essence       0.000133
Name: vehicule, dtype: float32

vehicules


vehicules        0.000625
electriques      0.000272
propres          0.000250
constructeurs    0.000245
hydrogene        0.000204
polluants        0.000204
diesel           0.000132
essence          0.000107
achat            0.000101
vehicule         0.000086
Name: vehicules, dtype: float32

taxes


taxes         0.000627
carburants    0.000187
augmenter     0.000105
baisser       0.000100
taxe          0.000075
transition    0.000074
impots        0.000072
produits      0.000072
taxer         0.000052
ecologique    0.000052
Name: taxes, dtype: float32

In [57]:
gc.collect()

126

# Question 9 :  Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?

In [59]:
question_9 = "Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?"
qu_9,df_qu_9_one_hot,df_cov_9,df_confidence_9,df_lift_9 = compute_similarities(question_9, new_res)

In [60]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_9[i].sort_values(ascending=False).head(10))

oui


oui    27145
non        0
Name: oui, dtype: uint16

non


non    18389
oui        0
Name: non, dtype: uint16

In [61]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_9[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [62]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_9[i].sort_values(ascending=False).head(10))

oui


oui    0.000037
non         NaN
Name: oui, dtype: float32

non


non    0.000054
oui         NaN
Name: non, dtype: float32

In [63]:
gc.collect()

14

# Question 10 :  Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?

In [64]:
question_10 = "Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?"
qu_10,df_qu_10_one_hot,df_cov_10,df_confidence_10,df_lift_10 = compute_similarities(question_10, new_res)

In [65]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_10[i].sort_values(ascending=False).head(10))

chauffage


chauffage     5666
mode          1506
changer        917
electrique     776
isolation      668
gaz            653
bois           583
ecologique     545
faire          505
aides          498
Name: chauffage, dtype: uint16

aide


aide            3202
financiere      1350
chauffage        452
chaudiere        312
installation     251
changer          245
chaleur          234
isolation        224
panneaux         217
pompe            192
Name: aide, dtype: uint16

aides


...

bois           1387
chauffage       583
poele           291
chaudiere       238
gaz             224
chauffe         219
chaleur         178
electrique      167
granules        151
electricite     139
Name: bois, dtype: uint16

electrique


electrique     1305
chauffage       776
tout            247
electricite     194
nucleaire       189
energie         184
gaz             177
bois            167
chaleur         141
changer         139
Name: electrique, dtype: uint16

installation


installation    1273
panneaux         334
chauffage        274
solaires         260
aide             251
chaleur          194
solaire          188
cout             155
aides            154
pompe            141
Name: installation, dtype: uint16

In [66]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_10[i].sort_values(ascending=False).head(10))

chauffage


chauffage     1.000000
mode          0.922229
collectif     0.749147
modes         0.699597
electrique    0.594636
systeme       0.507772
type          0.487805
changer       0.475869
ecologique    0.461864
systemes      0.450382
Name: chauffage, dtype: float32

aide


aide            1.000000
financiere      0.798817
fiscale         0.301939
revenus         0.256790
financement     0.241176
changement      0.199317
installation    0.197172
achat           0.189435
chaudiere       0.152717
credit          0.151141
Name: aide, dtype: float32

aides


...

bois           1.000000
poele          0.515044
granules       0.432665
chauffe        0.293960
electrique     0.127969
gaz            0.124791
chaudiere      0.116495
electricite    0.115641
maison         0.103673
ecologique     0.103390
Name: bois, dtype: float32

electrique


electrique     1.000000
nucleaire      0.366279
production     0.230453
tout           0.224342
electricite    0.161398
chauffage      0.136957
bois           0.120404
energie        0.104486
gaz            0.098607
maison         0.087347
Name: electrique, dtype: float32

installation


installation    1.000000
panneaux        0.164128
solaires        0.153031
cout            0.134666
solaire         0.099471
pompe           0.092459
chaleur         0.089401
aide            0.078389
prix            0.075574
energie         0.062465
Name: installation, dtype: float32

In [67]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_10[i].sort_values(ascending=False).head(10))

chauffage


chauffage     0.000176
mode          0.000163
collectif     0.000132
modes         0.000123
electrique    0.000105
systeme       0.000090
type          0.000086
changer       0.000084
ecologique    0.000082
systemes      0.000079
Name: chauffage, dtype: float32

aide


aide            0.000312
financiere      0.000249
fiscale         0.000094
revenus         0.000080
financement     0.000075
changement      0.000062
installation    0.000062
achat           0.000059
chaudiere       0.000048
credit          0.000047
Name: aide, dtype: float32

aides


...

bois           0.000721
poele          0.000371
granules       0.000312
chauffe        0.000212
electrique     0.000092
gaz            0.000090
chaudiere      0.000084
electricite    0.000083
maison         0.000075
ecologique     0.000075
Name: bois, dtype: float32

electrique


electrique     0.000766
nucleaire      0.000281
production     0.000177
tout           0.000172
electricite    0.000124
chauffage      0.000105
bois           0.000092
energie        0.000080
gaz            0.000076
maison         0.000067
Name: electrique, dtype: float32

installation


installation    0.000786
panneaux        0.000129
solaires        0.000120
cout            0.000106
solaire         0.000078
pompe           0.000073
chaleur         0.000070
aide            0.000062
prix            0.000059
energie         0.000049
Name: installation, dtype: float32

In [68]:
gc.collect()

147

# Question 11 : Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?

In [69]:
question_11 = "Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?"
qu_11,df_qu_11_one_hot,df_cov_11,df_confidence_11,df_lift_11 = compute_similarities(question_11, new_res)

In [70]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_11[i].sort_values(ascending=False).head(10))

oui


oui             19088
voiture             0
quotidiens          0
deplacements        0
non                 0
utilise             0
Name: oui, dtype: uint16

non


non             17924
voiture             0
quotidiens          0
deplacements        0
oui                 0
utilise             0
Name: non, dtype: uint16

voiture


...

voiture         9999
quotidiens      9999
deplacements    9999
utilise         9999
oui                0
non                0
Name: quotidiens, dtype: uint16

deplacements


voiture         9999
quotidiens      9999
deplacements    9999
utilise         9999
oui                0
non                0
Name: deplacements, dtype: uint16

utilise


voiture         9999
quotidiens      9999
deplacements    9999
utilise         9999
oui                0
non                0
Name: utilise, dtype: uint16

In [71]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_11[i].sort_values(ascending=False).head(10))

oui


oui             1.0
voiture         0.0
quotidiens      0.0
deplacements    0.0
non             0.0
utilise         0.0
Name: oui, dtype: float32

non


non             1.0
voiture         0.0
quotidiens      0.0
deplacements    0.0
oui             0.0
utilise         0.0
Name: non, dtype: float32

voiture


...

voiture         1.0
quotidiens      1.0
deplacements    1.0
utilise         1.0
oui             0.0
non             0.0
Name: quotidiens, dtype: float32

deplacements


voiture         1.0
quotidiens      1.0
deplacements    1.0
utilise         1.0
oui             0.0
non             0.0
Name: deplacements, dtype: float32

utilise


voiture         1.0
quotidiens      1.0
deplacements    1.0
utilise         1.0
oui             0.0
non             0.0
Name: utilise, dtype: float32

In [72]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_11[i].sort_values(ascending=False).head(10))

oui


oui             0.000052
utilise              NaN
non                  NaN
deplacements         NaN
quotidiens           NaN
voiture              NaN
Name: oui, dtype: float32

non


non             0.000056
utilise              NaN
oui                  NaN
deplacements         NaN
quotidiens           NaN
voiture              NaN
Name: non, dtype: float32

voiture


...

voiture         0.0001
quotidiens      0.0001
deplacements    0.0001
utilise         0.0001
non                NaN
oui                NaN
Name: quotidiens, dtype: float32

deplacements


voiture         0.0001
quotidiens      0.0001
deplacements    0.0001
utilise         0.0001
non                NaN
oui                NaN
Name: deplacements, dtype: float32

utilise


voiture         0.0001
quotidiens      0.0001
deplacements    0.0001
utilise         0.0001
non                NaN
oui                NaN
Name: utilise, dtype: float32

In [73]:
gc.collect()

217

# Question 12 :  Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?

In [74]:
question_12 = "Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?"
qu_12,df_qu_12_one_hot,df_cov_12,df_confidence_12,df_lift_12 = compute_similarities(question_12, new_res)

In [75]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_12[i].sort_values(ascending=False).head(10))

transports


transports    6604
commun        4500
voiture        861
developper     741
bus            693
velo           677
gratuite       656
cyclables      640
transport      624
communs        608
Name: transports, dtype: uint16

commun


commun        5934
transports    4500
transport     1854
voiture        887
velo           727
developper     631
bus            615
cyclables      579
pistes         559
utilise        497
Name: commun, dtype: uint16

transport


...

temps         1085
commun         382
voiture        365
transports     346
transport      320
trajet         277
travail        226
bus            207
velo           195
faire          135
Name: temps, dtype: uint16

solutions


solutions       1051
alternatives     427
transports       223
commun           205
voiture          192
transport        174
utilise          122
covoiturage      117
faut             114
velo             113
Name: solutions, dtype: uint16

electrique


electrique    1010
velo           583
voiture        277
achat          188
commun         160
transports     153
aide           143
vehicule       139
transport      126
bus            108
Name: electrique, dtype: uint16

In [76]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_12[i].sort_values(ascending=False).head(10))

transports


transports    1.000000
communs       0.929664
collectifs    0.850394
publics       0.789346
commun        0.758342
gratuite      0.718510
gratuits      0.715084
fiables       0.681275
frequents     0.620462
adaptes       0.546099
Name: transports, dtype: float32

commun


commun           1.000000
transports       0.681405
transport        0.593660
gratuite         0.498357
fiables          0.466135
gratuits         0.454376
gratuit          0.452436
developpement    0.448370
reseaux          0.446565
adaptes          0.432624
Name: commun, dtype: float32

transport


...

temps        1.000000
trajet       0.380495
travail      0.140635
prendre      0.139697
aller        0.139535
tout         0.127509
voiture      0.119711
faut         0.102496
transport    0.102466
faire        0.096017
Name: temps, dtype: float32

solutions


solutions       1.000000
alternatives    0.715243
faut            0.101604
covoiturage     0.076421
voiture         0.062971
utilise         0.062822
transport       0.055716
velo            0.036904
bus             0.036767
commun          0.034547
Name: solutions, dtype: float32

electrique


electrique    1.000000
achat         0.506739
aide          0.384409
vehicule      0.215170
velo          0.190398
voiture       0.090849
transport     0.040346
bus           0.039316
commun        0.026963
transports    0.023168
Name: electrique, dtype: float32

In [77]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_12[i].sort_values(ascending=False).head(10))

transports


transports    0.000151
communs       0.000141
collectifs    0.000129
publics       0.000120
commun        0.000115
gratuite      0.000109
gratuits      0.000108
fiables       0.000103
frequents     0.000094
adaptes       0.000083
Name: transports, dtype: float32

commun


commun           0.000169
transports       0.000115
transport        0.000100
gratuite         0.000084
fiables          0.000079
gratuits         0.000077
gratuit          0.000076
developpement    0.000076
reseaux          0.000075
adaptes          0.000073
Name: commun, dtype: float32

transport


...

temps        0.000922
trajet       0.000351
travail      0.000130
prendre      0.000129
aller        0.000129
tout         0.000118
voiture      0.000110
faut         0.000094
transport    0.000094
faire        0.000088
Name: temps, dtype: float32

solutions


solutions       0.000951
alternatives    0.000681
faut            0.000097
covoiturage     0.000073
voiture         0.000060
utilise         0.000060
transport       0.000053
velo            0.000035
bus             0.000035
commun          0.000033
Name: solutions, dtype: float32

electrique


electrique    0.000990
achat         0.000502
aide          0.000381
vehicule      0.000213
velo          0.000189
voiture       0.000090
transport     0.000040
bus           0.000039
commun        0.000027
transports    0.000023
Name: electrique, dtype: float32

In [78]:
gc.collect()

140

# Question 13 :   Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?

In [79]:
question_13 = "Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?"
qu_13,df_qu_13_one_hot,df_cov_13,df_confidence_13,df_lift_13 = compute_similarities(question_13, new_res)

In [80]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_13[i].sort_values(ascending=False).head(10))

commun


commun         21763
transports     21660
velo            9072
transport       5817
covoiturage     5749
demande         5612
auto            3507
partage         3500
marche           502
electrique       430
Name: commun, dtype: uint16

transports


transports     21761
commun         21660
velo            9053
covoiturage     5744
transport       5736
demande         5609
auto            3505
partage         3498
marche           502
electrique       428
Name: transports, dtype: uint16

velo


...

solution       325
travail          0
vehicule         0
commun           0
transport        0
pied             0
train            0
bus              0
demande          0
teletravail      0
Name: solution, dtype: uint16

electriques


electriques    283
transports     146
commun         137
velo           113
solutions        0
tout             0
vehicule         0
transport        0
pied             0
train            0
Name: electriques, dtype: uint16

tout


tout          277
commun        109
transports    108
solutions       0
travail         0
vehicule        0
transport       0
pied            0
train           0
bus             0
Name: tout, dtype: uint16

In [81]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_13[i].sort_values(ascending=False).head(10))

commun


commun         1.000000
transports     0.995359
covoiturage    0.736296
velo           0.705059
partage        0.692658
auto           0.692125
demande        0.674519
transport      0.671244
marche         0.671123
train          0.656250
Name: commun, dtype: float32

transports


transports     1.000000
commun         0.995267
covoiturage    0.735656
velo           0.703583
partage        0.692262
auto           0.691731
demande        0.674159
marche         0.671123
transport      0.661897
train          0.651442
Name: transports, dtype: float32

velo


...

solution       1.0
travail        0.0
vehicule       0.0
commun         0.0
transport      0.0
pied           0.0
train          0.0
bus            0.0
demande        0.0
teletravail    0.0
Name: solution, dtype: float32

electriques


electriques    1.000000
velo           0.008782
transports     0.006709
commun         0.006295
solutions      0.000000
tout           0.000000
vehicule       0.000000
transport      0.000000
pied           0.000000
train          0.000000
Name: electriques, dtype: float32

tout


tout          1.000000
commun        0.005009
transports    0.004963
solutions     0.000000
travail       0.000000
vehicule      0.000000
transport     0.000000
pied          0.000000
train         0.000000
bus           0.000000
Name: tout, dtype: float32

In [82]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_13[i].sort_values(ascending=False).head(10))

commun


commun         0.000046
transports     0.000046
covoiturage    0.000034
velo           0.000032
partage        0.000032
auto           0.000032
demande        0.000031
transport      0.000031
marche         0.000031
train          0.000030
Name: commun, dtype: float32

transports


transports     0.000046
commun         0.000046
covoiturage    0.000034
velo           0.000032
partage        0.000032
auto           0.000032
demande        0.000031
marche         0.000031
transport      0.000030
train          0.000030
Name: transports, dtype: float32

velo


...

solution       0.003077
auto                NaN
vehicule            NaN
commun              NaN
transport           NaN
pied                NaN
train               NaN
bus                 NaN
demande             NaN
teletravail         NaN
Name: solution, dtype: float32

electriques


electriques    0.003534
velo           0.000031
transports     0.000024
commun         0.000022
auto                NaN
vehicule            NaN
transport           NaN
pied                NaN
train               NaN
bus                 NaN
Name: electriques, dtype: float32

tout


tout          0.003610
commun        0.000018
transports    0.000018
auto               NaN
vehicule           NaN
transport          NaN
pied               NaN
train              NaN
bus                NaN
demande            NaN
Name: tout, dtype: float32

In [83]:
gc.collect()

147

# Question 14 :  Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?

In [84]:
question_14 = "Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?"
qu_14,df_qu_14_one_hot,df_cov_14,df_confidence_14,df_lift_14 = compute_similarities(question_14, new_res)

In [85]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_14[i].sort_values(ascending=False).head(10))

collectivites


collectivites    5188
locales          2318
territoriales    1418
entreprises       505
transports        394
commun            296
transport         223
region            202
solutions         200
communes          191
Name: collectivites, dtype: uint16

region


region           5137
departement      1563
commune           887
communes          439
transports        352
ville             298
commun            270
communaute        234
transport         206
collectivites     202
Name: region, dtype: uint16

departement


...

faire            1131
transports        177
tout              162
commun            147
faut              143
collectivites     119
solutions         116
transport         113
entreprises       111
voiture           106
Name: faire, dtype: uint16

departements


departements     1002
regions           477
communes          420
constructeurs       0
public              0
demande             0
tout                0
travail             0
moyens              0
commune             0
Name: departements, dtype: uint16

tous


tous             934
transports       127
collectivites    111
entreprises      106
public             0
demande            0
tout               0
travail            0
moyens             0
commune            0
Name: tous, dtype: uint16

In [86]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_14[i].sort_values(ascending=False).head(10))

collectivites


collectivites    1.000000
territoriales    0.953598
locales          0.855351
associations     0.274351
entreprises      0.218994
citoyens         0.210526
privees          0.186420
solutions        0.165153
transports       0.162340
place            0.156129
Name: collectivites, dtype: float32

region


region              1.000000
departement         0.453438
commune             0.283206
ville               0.237640
intercommunalite    0.234277
communaute          0.200686
niveau              0.195122
bus                 0.157601
sncf                0.145228
transports          0.145035
Name: region, dtype: float32

departement


...

faire            1.000000
faut             0.162316
voiture          0.158921
tout             0.132461
solutions        0.095789
commun           0.076923
transports       0.072930
transport        0.071248
entreprises      0.048135
collectivites    0.022938
Name: faire, dtype: float32

departements


departements     1.000000
regions          0.248308
communes         0.123059
constructeurs    0.000000
public           0.000000
demande          0.000000
tout             0.000000
travail          0.000000
moyens           0.000000
commune          0.000000
Name: departements, dtype: float32

tous


tous             1.000000
transports       0.052328
entreprises      0.045967
collectivites    0.021396
public           0.000000
demande          0.000000
tout             0.000000
travail          0.000000
moyens           0.000000
commune          0.000000
Name: tous, dtype: float32

In [87]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_14[i].sort_values(ascending=False).head(10))

collectivites


collectivites    0.000193
territoriales    0.000184
locales          0.000165
associations     0.000053
entreprises      0.000042
citoyens         0.000041
privees          0.000036
solutions        0.000032
transports       0.000031
place            0.000030
Name: collectivites, dtype: float32

region


region              0.000195
departement         0.000088
commune             0.000055
ville               0.000046
intercommunalite    0.000046
communaute          0.000039
niveau              0.000038
bus                 0.000031
sncf                0.000028
transports          0.000028
Name: region, dtype: float32

departement


...

faire            0.000884
faut             0.000144
voiture          0.000141
tout             0.000117
solutions        0.000085
commun           0.000068
transports       0.000064
transport        0.000063
entreprises      0.000043
collectivites    0.000020
Name: faire, dtype: float32

departements


departements    0.000998
regions         0.000248
communes        0.000123
voir                 NaN
locaux               NaN
mettre               NaN
publiques            NaN
territoire           NaN
mairies              NaN
pouvoirs             NaN
Name: departements, dtype: float32

tous


tous             0.001071
transports       0.000056
entreprises      0.000049
collectivites    0.000023
voir                  NaN
locaux                NaN
mettre                NaN
publiques             NaN
territoire            NaN
mairies               NaN
Name: tous, dtype: float32

In [88]:
gc.collect()

147

# Question 15 :  Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?

In [89]:
question_15 = "Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?"
qu_15,df_qu_15_one_hot,df_cov_15,df_confidence_15,df_lift_15 = compute_similarities(question_15, new_res)

In [90]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_15[i].sort_values(ascending=False).head(10))

exemple


exemple       9118
montrer       4482
donner        1987
pays          1816
france        1385
faire         1359
tout           600
prendre        594
ecologique     564
niveau         559
Name: exemple, dtype: uint16

pays


pays             7776
france           2144
exemple          1816
faire            1702
europe           1119
europeens         937
montrer           851
niveau            847
tous              821
environnement     818
Name: pays, dtype: uint16

faire


...

taxer            2022
produits          888
pays              548
france            337
importations      271
faire             270
exemple           263
environnement     252
non               208
pollueurs         196
Name: taxer, dtype: uint16

politique


politique        1925
pays              482
exemple           419
france            399
faire             375
ecologique        319
niveau            317
europeen          309
environnement     302
europe            298
Name: politique, dtype: uint16

europeens


europeens      1838
pays            937
france          501
faire           404
exemple         388
europe          277
niveau          251
tous            241
europeen        217
partenaires     200
Name: europeens, dtype: uint16

In [91]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_15[i].sort_values(ascending=False).head(10))

exemple


exemple      1.000000
montrer      0.804668
donner       0.695485
nordiques    0.431487
attendre     0.427208
prendre      0.420382
discours     0.413793
loin         0.397229
commencer    0.374656
devenir      0.366450
Name: exemple, dtype: float32

pays


pays         1.000000
nordiques    0.956268
nord         0.730887
avance       0.688442
inspirer     0.604131
europeens    0.509793
rapport      0.501529
membres      0.496063
efforts      0.494983
retard       0.488525
Name: pays, dtype: float32

faire


...

taxer           1.000000
importations    0.516190
fortement       0.496350
importes        0.420354
etrangers       0.412195
respectent      0.401345
produits        0.332087
polluants       0.292434
respectant      0.289474
camions         0.280000
Name: taxer, dtype: float32

politique


politique           1.000000
commune             0.642241
agricole            0.532319
vraie               0.524917
environnementale    0.442149
faveur              0.376667
adopter             0.318612
protection          0.267442
energetique         0.223801
europeenne          0.179366
Name: politique, dtype: float32

europeens


europeens      1.000000
partenaires    0.361664
voisins        0.211129
etats          0.121212
pays           0.120499
allemagne      0.116379
tous           0.113787
convaincre     0.111209
francais       0.105882
proposer       0.103933
Name: europeens, dtype: float32

In [92]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_15[i].sort_values(ascending=False).head(10))

exemple


exemple      0.000110
montrer      0.000088
donner       0.000076
nordiques    0.000047
attendre     0.000047
prendre      0.000046
discours     0.000045
loin         0.000044
commencer    0.000041
devenir      0.000040
Name: exemple, dtype: float32

pays


pays         0.000129
nordiques    0.000123
nord         0.000094
avance       0.000089
inspirer     0.000078
europeens    0.000066
rapport      0.000064
membres      0.000064
efforts      0.000064
retard       0.000063
Name: pays, dtype: float32

faire


...

taxer           0.000495
importations    0.000255
fortement       0.000245
importes        0.000208
etrangers       0.000204
respectent      0.000198
produits        0.000164
polluants       0.000145
respectant      0.000143
camions         0.000138
Name: taxer, dtype: float32

politique


politique           0.000519
commune             0.000334
agricole            0.000277
vraie               0.000273
environnementale    0.000230
faveur              0.000196
adopter             0.000166
protection          0.000139
energetique         0.000116
europeenne          0.000093
Name: politique, dtype: float32

europeens


europeens      0.000544
partenaires    0.000197
voisins        0.000115
etats          0.000066
pays           0.000066
allemagne      0.000063
tous           0.000062
convaincre     0.000061
francais       0.000058
proposer       0.000057
Name: europeens, dtype: float32

In [ ]:
gc.collect()

105

# Question 16 :   Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?

In [ ]:
question_15 = "Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?"
qu_15,df_qu_15_one_hot,df_cov_15,df_confidence_15,df_lift_15 = compute_similarities(question_15, new_res)

In [ ]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_15[i].sort_values(ascending=False).head(10))

In [ ]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_15[i].sort_values(ascending=False).head(10))

In [ ]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_15[i].sort_values(ascending=False).head(10))

In [ ]:
gc.collect()